In [64]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy import wcs
from astropy.table import Table
from astropy.io import fits

from scipy.stats import truncnorm
from skimage.util import view_as_windows

In [65]:
PATH_DATA = '/Volumes/ES-LaCie/Documents/MATLAS'
PATH_SAVE = '/Volumes/ES-LaCie/Documents/MATLAS/Data/r_band'